In [2]:
import sys
from astroquery.gaia import Gaia

In [3]:
tables = Gaia.load_tables(only_names=True)

INFO: Retrieving tables... [astroquery.utils.tap.core]
INFO: Parsing tables... [astroquery.utils.tap.core]
INFO: Done. [astroquery.utils.tap.core]


In [4]:
for table in tables:
    print(table.name)

external.apassdr9
external.catwise2020
external.gaiadr2_astrophysical_parameters
external.gaiadr2_geometric_distance
external.gaiaedr3_distance
external.gaiaedr3_gcns_main_1
external.gaiaedr3_gcns_rejected_1
external.gaiaedr3_spurious
external.gaia_eso_survey
external.galex_ais
external.lamost_dr9_lrs
external.lamost_dr9_mrs
external.ravedr5_com
external.ravedr5_dr5
external.ravedr5_gra
external.ravedr5_on
external.ravedr6
external.sdssdr13_photoprimary
external.skymapperdr1_master
external.skymapperdr2_master
external.tmass_xsc
external.xgboost_table1
external.xgboost_table2
gaiadr1.aux_qso_icrf2_match
gaiadr1.ext_phot_zero_point
gaiadr1.allwise_best_neighbour
gaiadr1.allwise_neighbourhood
gaiadr1.gsc23_best_neighbour
gaiadr1.gsc23_neighbourhood
gaiadr1.ppmxl_best_neighbour
gaiadr1.ppmxl_neighbourhood
gaiadr1.sdss_dr9_best_neighbour
gaiadr1.sdss_dr9_neighbourhood
gaiadr1.tmass_best_neighbour
gaiadr1.tmass_neighbourhood
gaiadr1.ucac4_best_neighbour
gaiadr1.ucac4_neighbourhood
gaiadr1.u

In [6]:
meta = Gaia.load_table('gaiadr2.gaia_source')
meta

In [7]:
print(meta)

TAP Table name: gaiadr2.gaia_source
Description: This table has an entry for every Gaia observed source as listed in the
Main Database accumulating catalogue version from which the catalogue
release has been generated. It contains the basic source parameters,
that is only final data (no epoch data) and no spectra (neither final
nor epoch).
Size (bytes): 4906520690688
Num. columns: 96


In [8]:
# Printing column names 
for column in meta.columns:
    print(column.name)

solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrometric_n_obs_al
astrometric_n_obs_ac
astrometric_n_good_obs_al
astrometric_n_bad_obs_al
astrometric_gof_al
astrometric_chi2_al
astrometric_excess_noise
astrometric_excess_noise_sig
astrometric_params_solved
astrometric_primary_flag
astrometric_weight_al
astrometric_pseudo_colour
astrometric_pseudo_colour_error
mean_varpi_factor_al
astrometric_matched_observations
visibility_periods_used
astrometric_sigma5d_max
frame_rotator_object_type
matched_observations
duplicated_source
phot_g_n_obs
phot_g_mean_flux
phot_g_mean_flux_error
phot_g_mean_flux_over_error
phot_g_mean_mag
phot_bp_n_obs
phot_bp_mean_flux
phot_bp_mean_flux_error
phot_bp_mean_flux_over_error
phot_bp_mean_ma

Writing Queries

In [9]:
query1 = """SELECT 
TOP 10
source_id, ra, dec, parallax 
FROM gaiadr2.gaia_source
"""

To run this query, we use the Gaia object, which represents our connection to the Gaia database, and invoke launch_job:

In [12]:
job = Gaia.launch_job(query1)
job

The result is an object that represents the job running on a Gaia server.

If you print it, it displays metadata for the forthcoming results.

In [11]:
print(job)

<Table length=10>
   name    dtype  unit                            description                             n_bad
--------- ------- ---- ------------------------------------------------------------------ -----
source_id   int64      Unique source identifier (unique within a particular Data Release)     0
       ra float64  deg                                                    Right ascension     0
      dec float64  deg                                                        Declination     0
 parallax float64  mas                                                           Parallax     1
Jobid: None
Phase: COMPLETED
Owner: None
Output file: d8e602f7-0d8c-11f1-91f6-bc97e148b76b-O-result.vot.gz
Results: None


In [13]:
results = job.get_results()
type(results)

astropy.table.table.Table

table is repeadted 3 times, the first is the name of the module, the second is the name of the submodule, and the third is the name of the class. Most of the time we only care about the last one

In [15]:
results

source_id,ra,dec,parallax
,deg,deg,mas
int64,float64,float64,float64
4464185571489765376,246.0771571149933,15.475361948153898,0.3581981880000272
2021650318958976512,294.93613682293886,25.320312631163834,0.4482071426928564
6003882045187270400,229.28259303837524,-42.97826091593811,0.3814393435896097
6680659523978470144,306.5616715416606,-40.54589081856055,-2.227432561581064
6680733225618222592,307.6583019373705,-40.00379241243431,2.2595484953413174
5800583066773602560,224.61633794146624,-67.07005997802717,--
5781838313353791232,236.48568321530968,-75.24915473149034,0.12674646527678737
4144471714989233408,268.5454915814623,-17.682461147100533,0.12228975502535505


Asynchronous queries

launch_job asks the server to run the job "synchronously", which normally means it runs immediately. But synchronous jobs are limited to 2000 rows. For queries that return more rows, you should run "asynchronously", which mean they might take longer to get started.

In [16]:
query2 = """SELECT 
TOP 3000
source_id, ra, dec, pmra, pmdec, parallax
FROM gaiadr2.gaia_source
WHERE parallax < 1
"""

In [17]:
job = Gaia.launch_job_async(query2)
job

INFO: Query finished. [astroquery.utils.tap.core]


In [18]:
results = job.get_results()
results

source_id,ra,dec,pmra,pmdec,parallax
,deg,deg,mas / yr,mas / yr,mas
int64,float64,float64,float64,float64,float64
4464194092703698048,243.32130865040432,13.330617936122106,1.7000647673456717,-5.321352978282461,0.9563165300756201
4144478685704350848,268.8294381776294,-17.715958699116342,-2.2818304891065475,-3.8542916801236102,-0.13824660844890915
6680743735401494400,306.7980238147647,-40.443398866341624,5.807358244547909,-7.7137197560782145,0.6907975270201289
5781895079941575936,236.88037348536776,-74.9481677176003,-1.1827249541226612,0.12138999682734725,0.8768701905324682
4464097408697203584,246.6563063551879,15.029986190975974,-0.6239380634092571,-3.523464869817627,0.2817961793704932
6680652510296877312,306.46669507233855,-40.70136983370839,3.5414420559096906,-4.612168641081958,0.47483426400353784
6680606914923508864,306.43046791618684,-41.13172505866114,-9.72374779425799,-6.084057701704904,0.05426492746635359
2047223447577020800,294.9144562493088,34.24853438831421,-4.687901021064943,-6.633834457420785,0.18115095707888837


Excercise:
Modify the previous query to select rows where bp_rp is between -0.75 and 2.

In [20]:
query2 = """SELECT 
TOP 3000
source_id, ra, dec, pmra, pmdec, parallax, bp_rp
FROM gaiadr2.gaia_source
WHERE parallax < 1
AND bp_rp BETWEEN 0.75 AND 2
"""

In [21]:
job = Gaia.launch_job_async(query2)
results = job.get_results()
results

INFO: Query finished. [astroquery.utils.tap.core]


source_id,ra,dec,pmra,pmdec,parallax,bp_rp
,deg,deg,mas / yr,mas / yr,mas,mag
int64,float64,float64,float64,float64,float64,float32
4464174164056625024,246.29390759706718,15.47257793268518,4.866522777608916,-15.68232420624172,0.4746433256414075,1.8373985
4464064419051281024,246.39883051914418,14.80697876478872,-5.893077730502974,0.2984844583935275,-1.4422448853355199,1.2348518
4464088372086010624,246.51526933453388,14.871581300604223,-0.891411042702642,-12.287330572563171,0.7082992863586364,1.5341034
5781859277093847168,235.98950374334467,-75.37000472177178,-4.329144102355594,1.2846164519981857,0.27512503373236563,0.96554756
6725810281803795328,273.52271376498413,-40.12976882370781,-5.121213912805766,-9.715994435328657,0.05761870932168217,1.0488033
6725818734303630080,273.74055299583915,-39.972055208376815,0.9867391379887143,-3.9779711186990387,0.4921957943715942,0.8244209
1872308293103643392,316.05153587957994,39.34991504418823,-4.692647677166686,-7.030073361885714,0.6103225286955192,1.5929699
2047227914343609856,294.8196071319395,34.34691358182021,-1.5495411121185547,-4.149763248993786,0.27846272263872673,1.1116066


bp_rp contains BP-RP color, which is the difference between two other columns, phot_bp_mean_mag and phot_rp_mean_mag

Selecting stars with bp-rp less than 2 excludes many class M dwarf stars, which are low temperature, low luminosity. A star like that at GD-1's distance would be hard to detect, so if it is detected, it it more likely to be in the foreground.

Formatting Queries

In [27]:
columns = 'source_id, ra, dec, pmra, pmdec, parallax, bp_rp'

In [28]:
query3_base = """SELECT 
TOP 10 
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2
"""

In [29]:
# Assemble the query by invoking the format method on teh base string
query3 = query3_base.format(columns=columns)
query3

'SELECT \nTOP 10 \nsource_id, ra, dec, pmra, pmdec, parallax, bp_rp\nFROM gaiadr2.gaia_source\nWHERE parallax < 1\n  AND bp_rp BETWEEN -0.75 AND 2\n'

In [30]:
print(query3)

SELECT 
TOP 10 
source_id, ra, dec, pmra, pmdec, parallax, bp_rp
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2



In [31]:
job = Gaia.launch_job(query3)
results = job.get_results()
results

source_id,ra,dec,pmra,pmdec,parallax,bp_rp
,deg,deg,mas / yr,mas / yr,mas,mag
int64,float64,float64,float64,float64,float64,float32
4464174164056625024,246.29390759706718,15.47257793268518,4.866522777608916,-15.68232420624172,0.4746433256414075,1.8373985
4464064419051281024,246.39883051914418,14.80697876478872,-5.893077730502974,0.2984844583935275,-1.4422448853355199,1.2348518
4464088372086010624,246.51526933453388,14.871581300604223,-0.891411042702642,-12.287330572563171,0.7082992863586364,1.5341034
5781859277093847168,235.98950374334467,-75.37000472177178,-4.329144102355594,1.2846164519981857,0.27512503373236563,0.96554756
6725810281803795328,273.52271376498413,-40.12976882370781,-5.121213912805766,-9.715994435328657,0.05761870932168217,1.0488033
6725818734303630080,273.74055299583915,-39.972055208376815,0.9867391379887143,-3.9779711186990387,0.4921957943715942,0.8244209
1872308293103643392,316.05153587957994,39.34991504418823,-4.692647677166686,-7.030073361885714,0.6103225286955192,1.5929699
2047227914343609856,294.8196071319395,34.34691358182021,-1.5495411121185547,-4.149763248993786,0.27846272263872673,1.1116066


Excercise:
Modify query3_base to replace 1 with a format specifier like {max_parallax}. Now, when you call format, add a keyword argument that assigns a value to max_parallax, and confirm that the format specifier gets replaced with the value you provide.

In [32]:
max_parallax = 0.7

query3_base = """SELECT 
TOP 10 
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < {max_parallax}
  AND bp_rp BETWEEN -0.75 AND 2
"""

In [33]:
query3 = query3_base.format(columns=columns, max_parallax = max_parallax)
print(query3)

SELECT 
TOP 10 
source_id, ra, dec, pmra, pmdec, parallax, bp_rp
FROM gaiadr2.gaia_source
WHERE parallax < 0.7
  AND bp_rp BETWEEN -0.75 AND 2



In [34]:
job = Gaia.launch_job(query3)
results = job.get_results()
results

source_id,ra,dec,pmra,pmdec,parallax,bp_rp
,deg,deg,mas / yr,mas / yr,mas,mag
int64,float64,float64,float64,float64,float64,float32
4464064281612307328,246.36320903464275,14.786967077000254,2.5965490719307223,-3.6292820144214915,-0.46443857493205865,1.785284
5781849896885018880,235.19975233566362,-75.55468578963749,-1.9710723531051824,-4.372155202507062,0.4253216182937757,1.2947617
5781901539572606464,235.51724175589783,-75.06158382985825,-2.6423192861858174,-4.739368820984588,0.3152221668527339,0.8306112
4144471405749384576,268.6199001702355,-17.659064277198986,-2.795973995276563,-4.1423019546991675,-0.16340868487043947,1.7634106
6003883209118221184,229.22494454302014,-42.92405238318647,-6.184867004691714,-4.124304543749411,0.3950098960922301,0.88838196
1872283859029096960,316.55905648269885,39.289918100563675,-5.382498712551123,-3.8864336862991276,0.25597099907150767,1.3747959
2047240421291981056,294.3109125784004,34.00465903208172,-5.273240273968838,-6.783737648097816,0.059687003855267,0.78653336
2065078653124845568,313.3269648288874,39.98048972298352,-4.107636321058465,2.4035577429869703,0.4058079273735532,1.7896366


In [37]:
print(f"Max parallax in results: {results["parallax"].max()}")

Max parallax in results: 0.6682033551752734
